In [0]:
%fs cp /FileStore/tables/data.csv /FileStore/tables/autoloader_input/data1.csv

In [0]:
from pyspark.sql.functions import input_file_name, current_timestamp

raw_path = "/FileStore/tables/autoloader_input/"

df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")       # File format: CSV
    .option("header", "true")                 # First row as header
    .option("inferSchema", "true")            # Infer schema automatically
    .option("cloudFiles.schemaLocation", "/FileStore/tables/_schemas/bronze/")  # Schema location
    .load(raw_path)                           # Input directory
    .withColumn("ingest_time", current_timestamp())  # Add ingestion timestamp
    .withColumn("source_file", input_file_name())    # Add source file name
)

# Write stream to Bronze Table
df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/FileStore/tables/_checkpoints/bronze/") \
    .option("mergeSchema", "true") \
    .table("retail_demo.bronze_sales")